Import Library

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf

from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



Read CSV File

In [2]:
df = pd.read_csv('tweet.csv')
df.head()

,Abusive,Tweet
0,1,cowok usaha lacak perhati gue lantas remeh per...
1,1,telat tau edan sarap gue gaul cigax jifla cal ...
2,0,41 kadang pikir percaya tuhan jatuh kali kali ...
3,0,ku tau mata sipit lihat
4,1,kaum cebong kafir lihat dongok dungu haha


Drop Missing Rows

In [3]:
# drop missing rows
df.dropna(axis=0, inplace=True)

Print Lenght of Data

In [4]:
text = df["Tweet"].tolist()
print(len(text))

13121


Make it to Categorical

In [5]:
y = df["Abusive"]
y = to_categorical(y)
print(y)
#0 itu negatif, 1 itu positif

[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


Count Data Each Categorical

In [6]:
df["Abusive"].value_counts()

0    8088
1    5033
Name: Abusive, dtype: int64

Do Tokenizer

In [7]:
token = Tokenizer()
token.fit_on_texts(text)

In [8]:
# if you want to print tokenizer word, run code below 
# token.index_word 

Print Lenght of Index of Word

In [9]:
vocab = len(token.index_word)+1
print(vocab)

13268


Test Text to Tokenize Index

In [10]:
x = ['sinting kau ya']
token.texts_to_sequences(x)

[[558, 1035, 8]]

Encode Every Each Tweet Dataset

In [11]:
encode_text = token.texts_to_sequences(text)
# if you want to print every tokenizer tweet
# print(encode_text)

Do Padding Every Encode Tweet Dataset

In [12]:
max_kata = 100
x=pad_sequences(encode_text,maxlen = max_kata, padding="post")
print(x)

[[ 324  161 3546 ...    0    0    0]
 [1908   49  464 ...    0    0    0]
 [3547  598  101 ...    0    0    0]
 ...
 [  66   66  376 ...    0    0    0]
 [ 111 2819  291 ...    0    0    0]
 [ 569  325    8 ...    0    0    0]]


# **80 20 ratio**
Performing learning for 80% data training and 20% data testing.

Split data test and test test

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1, test_size = 0.2, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

Change to Data to Array

In [14]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

Define Model

In [15]:
vec_size = 100

model = tf.keras.Sequential()
model.add(Embedding(vocab,vec_size,input_length=max_kata))
model.add(Conv1D(64,3,activation='relu'))

model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1326800   
                                                                 
 conv1d (Conv1D)             (None, 98, 64)            19264     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1,346,194
Trainable params: 1,346,194
Non-trainable params: 0
______________________________________________

In [16]:
from keras.metrics import Precision, Recall
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['accuracy', Precision(), Recall()])

In [17]:
model.fit(x_train,y_train, epochs=10, validation_data =(x_test,y_test))

Epoch 1/10
263/263 [==============================] - 9s 32ms/step - loss: 0.4459 - accuracy: 0.7833 - precision: 0.7833 - recall: 0.7833 - val_loss: 0.2179 - val_accuracy: 0.9166 - val_precision: 0.9166 - val_recall: 0.9166
Epoch 2/10
263/263 [==============================] - 9s 33ms/step - loss: 0.1781 - accuracy: 0.9390 - precision: 0.9390 - recall: 0.9390 - val_loss: 0.1945 - val_accuracy: 0.9253 - val_precision: 0.9253 - val_recall: 0.9253
Epoch 3/10
263/263 [==============================] - 8s 30ms/step - loss: 0.1120 - accuracy: 0.9639 - precision: 0.9639 - recall: 0.9639 - val_loss: 0.2150 - val_accuracy: 0.9204 - val_precision: 0.9204 - val_recall: 0.9204
Epoch 4/10
263/263 [==============================] - 8s 32ms/step - loss: 0.0726 - accuracy: 0.9768 - precision: 0.9768 - recall: 0.9768 - val_loss: 0.2453 - val_accuracy: 0.9208 - val_precision: 0.9208 - val_recall: 0.9208
Epoch 5/10
263/263 [==============================] - 8s 30ms/step - loss: 0.0516 - accuracy: 0.9857

Evaluate and print Accuracy

In [18]:
import keras.backend as K

def f1_score(precision, recall):
    ''' Function to calculate f1 score '''
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [19]:
# Evaluate model on the test set
loss, accuracy, precision, recall = model.evaluate(x_test, y_test, verbose=0)
# Print metrics
print('')
print('Accuracy  : {:.4f}'.format(accuracy))
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))


Accuracy  : 0.9044
Precision : 0.9044
Recall    : 0.9044
F1 Score  : 0.9044


Get Encode of Predict Data

In [20]:
def get_encode(x):
  x = token.texts_to_sequences(x)
  x = pad_sequences(x,maxlen = max_kata, padding = "post")
  return x

Get Sentiment Classesof Predict Data

In [21]:
def get_sentiment_classes(x):
  x = get_encode(x)
  predict_x=model.predict(x) 
  classes_x=np.argmax(predict_x,axis=1)
  sentiment_classes = ['tidak kasar','kasar']
  print('kata tersebut mengandung konotasi',sentiment_classes[classes_x[0]])

Predict Data 1

In [22]:
# untuk melakukan prediksi kata yang tidak kasar 
get_sentiment_classes(['ibu peri hari ini cantik banget ya'])

kata tersebut mengandung konotasi tidak kasar


Predict Data 2

In [23]:
# untuk melakukan prediksi kata yang kasar
get_sentiment_classes(['bangsat cok raimu koyok asu'])

kata tersebut mengandung konotasi kasar


# **70 30 ratio**
Performing learning for 70% data training and 30% data testing. 


Split data test and test test

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1, test_size = 0.3, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=1)

Change to Data to Array

In [25]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

Define Model

In [26]:
vec_size = 100

model = tf.keras.Sequential()
model.add(Embedding(vocab,vec_size,input_length=max_kata))
model.add(Conv1D(64,3,activation='relu'))

model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1326800   
                                                                 
 conv1d_1 (Conv1D)           (None, 98, 64)            19264     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,346,194
Trainable params: 1,346,194
Non-trainable params: 0
____________________________________________

In [27]:
from keras.metrics import Precision, Recall
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['accuracy', Precision(), Recall()])

In [28]:
model.fit(x_train,y_train, epochs=10, validation_data =(x_test,y_test))

Epoch 1/10
201/201 [==============================] - 8s 35ms/step - loss: 0.4850 - accuracy: 0.7629 - precision_1: 0.7629 - recall_1: 0.7629 - val_loss: 0.2476 - val_accuracy: 0.9126 - val_precision_1: 0.9126 - val_recall_1: 0.9126
Epoch 2/10
201/201 [==============================] - 6s 32ms/step - loss: 0.1850 - accuracy: 0.9367 - precision_1: 0.9367 - recall_1: 0.9367 - val_loss: 0.2179 - val_accuracy: 0.9180 - val_precision_1: 0.9180 - val_recall_1: 0.9180
Epoch 3/10
201/201 [==============================] - 6s 32ms/step - loss: 0.1056 - accuracy: 0.9652 - precision_1: 0.9652 - recall_1: 0.9652 - val_loss: 0.2358 - val_accuracy: 0.9141 - val_precision_1: 0.9141 - val_recall_1: 0.9141
Epoch 4/10
201/201 [==============================] - 6s 32ms/step - loss: 0.0651 - accuracy: 0.9795 - precision_1: 0.9795 - recall_1: 0.9795 - val_loss: 0.2630 - val_accuracy: 0.9131 - val_precision_1: 0.9131 - val_recall_1: 0.9131
Epoch 5/10
201/201 [==============================] - 7s 33ms/step -

Evaluate and print Accuracy

In [29]:
import keras.backend as K

def f1_score(precision, recall):
    ''' Function to calculate f1 score '''
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [30]:
# Evaluate model on the test set
loss, accuracy, precision, recall = model.evaluate(x_test, y_test, verbose=0)
# Print metrics
print('')
print('Accuracy  : {:.4f}'.format(accuracy))
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))


Accuracy  : 0.9098
Precision : 0.9098
Recall    : 0.9098
F1 Score  : 0.9098


Get Encode of Predict Data

In [31]:
def get_encode(x):
  x = token.texts_to_sequences(x)
  x = pad_sequences(x,maxlen = max_kata, padding = "post")
  return x

Get Sentiment Classesof Predict Data

In [32]:
def get_sentiment_classes(x):
  x = get_encode(x)
  predict_x=model.predict(x) 
  classes_x=np.argmax(predict_x,axis=1)
  sentiment_classes = ['tidak kasar','kasar']
  print('kata tersebut mengandung konotasi',sentiment_classes[classes_x[0]])

Predict Data 1

In [33]:
# untuk melakukan prediksi kata yang tidak kasar 
get_sentiment_classes(['ibu peri hari ini cantik banget ya'])

kata tersebut mengandung konotasi tidak kasar


Predict Data 2

In [34]:
# untuk melakukan prediksi kata yang kasar
get_sentiment_classes(['bangsat cok raimu koyok asu'])

kata tersebut mengandung konotasi kasar


# **60 40 ratio**
Performing learning for 60% data training and 40% data testing.

Split data test and test test

In [35]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1, test_size = 0.4, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.4, random_state=1)

Change to Data to Array

In [36]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

Define Model

In [37]:
vec_size = 100

model = tf.keras.Sequential()
model.add(Embedding(vocab,vec_size,input_length=max_kata))
model.add(Conv1D(64,3,activation='relu'))

model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          1326800   
                                                                 
 conv1d_2 (Conv1D)           (None, 98, 64)            19264     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,346,194
Trainable params: 1,346,194
Non-trainable params: 0
____________________________________________

In [38]:
from keras.metrics import Precision, Recall
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['accuracy', Precision(), Recall()])

In [39]:
model.fit(x_train,y_train, epochs=10, validation_data =(x_test,y_test))

Epoch 1/10
148/148 [==============================] - 7s 41ms/step - loss: 0.5592 - accuracy: 0.7025 - precision_2: 0.7025 - recall_2: 0.7025 - val_loss: 0.3631 - val_accuracy: 0.8617 - val_precision_2: 0.8617 - val_recall_2: 0.8617
Epoch 2/10
148/148 [==============================] - 5s 34ms/step - loss: 0.2312 - accuracy: 0.9176 - precision_2: 0.9176 - recall_2: 0.9176 - val_loss: 0.2318 - val_accuracy: 0.9126 - val_precision_2: 0.9126 - val_recall_2: 0.9126
Epoch 3/10
148/148 [==============================] - 5s 34ms/step - loss: 0.1150 - accuracy: 0.9613 - precision_2: 0.9613 - recall_2: 0.9613 - val_loss: 0.2498 - val_accuracy: 0.9118 - val_precision_2: 0.9118 - val_recall_2: 0.9118
Epoch 4/10
148/148 [==============================] - 6s 38ms/step - loss: 0.0684 - accuracy: 0.9812 - precision_2: 0.9812 - recall_2: 0.9812 - val_loss: 0.2756 - val_accuracy: 0.9091 - val_precision_2: 0.9091 - val_recall_2: 0.9091
Epoch 5/10
148/148 [==============================] - 5s 34ms/step -

Evaluate and print Accuracy

In [40]:
import keras.backend as K

def f1_score(precision, recall):
    ''' Function to calculate f1 score '''
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [41]:
# Evaluate model on the test set
loss, accuracy, precision, recall = model.evaluate(x_test, y_test, verbose=0)
# Print metrics
print('')
print('Accuracy  : {:.4f}'.format(accuracy))
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))


Accuracy  : 0.8988
Precision : 0.8988
Recall    : 0.8988
F1 Score  : 0.8988


Get Encode of Predict Data

In [42]:
def get_encode(x):
  x = token.texts_to_sequences(x)
  x = pad_sequences(x,maxlen = max_kata, padding = "post")
  return x

Get Sentiment Classesof Predict Data

In [43]:
def get_sentiment_classes(x):
  x = get_encode(x)
  predict_x=model.predict(x) 
  classes_x=np.argmax(predict_x,axis=1)
  sentiment_classes = ['tidak kasar','kasar']
  print('kata tersebut mengandung konotasi',sentiment_classes[classes_x[0]])

Predict Data 1

In [44]:
# untuk melakukan prediksi kata yang tidak kasar 
get_sentiment_classes(['ibu peri hari ini cantik banget ya'])

kata tersebut mengandung konotasi kasar


Predict Data 2

In [45]:
# untuk melakukan prediksi kata yang kasar
get_sentiment_classes(['bangsat cok raimu koyok asu'])

kata tersebut mengandung konotasi kasar
